In [1]:
%autosave 0
#!/usr/bin/env python
import argparse
import os
import sys
import pandas as pd
import patsy
import json
import numpy as np
from create_flame_model_files import create_flame_model_files
__version__ = 0.1

Autosave disabled


1. 'bids_dir', 'The directory with the input dataset           'formatted according to the BIDS standard.'
2. 'output_dir', 'The directory where the output files '        'should be stored. If you are running group level analysis '   'this folder should be prepopulated with the results of the'  'participant level analysis.'
3. 'working_dir', 'The directory where intermediary files '          'are stored while working on them.'
4. 'analysis_level', 'Level of the analysis that will be performed.  'Multiple participant level analyses can be run independently '     '(in parallel using the same output_dir. Use test_model to         'the model and contrast files, but not run the anlaysis.',         choices=['participant', 'group', 'test_model']
5. 'model_file', 'JSON file describing the model and contrasts'  'that should be.'
'--num_iterations', 'Number of iterations used by randomise.',      default=10000, type=int
'--num_processors', 'Number of processors used at a time for randomise',default=1, type=int'-v', '--version', action='version', version='BIDS-App example version {}'.format(__version__)


In [21]:
in_model_file = '/home/jovyan/work/test_data/IBA_TRT/model.json'
in_bids_dir =  '/home/jovyan/work/test_data/IBA_TRT/'
model_files_outdir = '/home/jovyan/work/test_data/'

In [4]:
pwd

'/home/jovyan/work'

In [5]:
# load in the model
with open(in_model_file) as model_fd:
     model_dict = json.load(model_fd)

In [6]:
# parse the model string to determine which columns of the pheno
    # file we are interested in
in_columns = model_dict["model"].replace("-1", "").replace("-", "+").split("+")
t_columns = []
for column in in_columns:
    if '*' in column:
            t_columns += column.split("*")
    else:
            t_columns.append(column)
    in_columns = list(set(t_columns))

In [7]:
# read in the pheno file
pheno_df = pd.read_csv(os.path.join(in_bids_dir,'participants.tsv'), sep='\t')

# reduce the file to just the columns that we are interested in
#heno_df = pheno_df[['participant_id'] + in_columns]
pheno_df = pheno_df[['sub'] + in_columns]

# remove rows that have empty elements
pheno_df = pheno_df.dropna()

In [8]:
pheno_df.tail()

,sub,age,sex
45,sub-27255,22,M
46,sub-27256,33,F
47,sub-27256,33,F
48,sub-27257,40,M
49,sub-27258,21,F


In [9]:
%%capture
# go through data, verify that we can find a corresponding entry in
# the pheno file, and keep track of the indices so that we can
    # reorder the pheno to correspond
t_file_list = []
pheno_key_list = []

for root, dirs, files in os.walk(in_bids_dir):
    for filename in files:
    
        if not filename.endswith(".nii.gz"):
            continue

        # make a dictionary from the key-value chunks
        f_chunks = (filename.split(".")[0]).split("_")
        f_dict = {chunk.split("-")[0]: "-".join(chunk.split("-")[1:]) for chunk in f_chunks[:-1]}
        
        if not f_dict['ses']:
            f_dict['ses'] = '1'
        x = np.array(f_dict['sub'], dtype='|S7')
        y = x.astype(np.int)
        y = str(y)
        f_participant_id = "_".join(["-".join(["sub", y]), "-".join(["ses", f_dict["ses"]])])
        
        # find the row of the pheno_df that corresponds to the file and save it to pheno_key_list
        participant_index = [index for index, file_id in enumerate(pheno_df["sub"])
                                 if file_id in f_participant_id]
        if len(participant_index) == 0:
            print("Could not find entry in phenotype file for {0}, dropping it.".format(
                    os.path.join(root, filename)))
        elif len(participant_index) > 0:      
        
            pheno_key_list.append(participant_index[0])
            t_file_list.append(os.path.join(root, filename))
        else:
            raise ValueError("Found multiple entries for {0} in {1}", f_participant_id,
                                 os.path.join(in_bids_dir, 'participants.tsv'))

In [10]:
pheno_df.head()

,sub,age,sex
0,sub-27223,20,M
1,sub-27224,40,F
2,sub-27224,40,F
3,sub-27225,26,M
4,sub-27225,26,M


In [11]:
# now create the design.mat file

# remove participant_id column
pheno_df = pheno_df[in_columns]

# reduce to the rows that we are using, and reorder to match the file list
pheno_df = pheno_df.iloc[pheno_key_list, :]

print ("{0} rows in design matrix".format(len(pheno_df.index)))

# de-mean all numeric columns, we expect categorical variables to be encoded with strings
for df_ndx in pheno_df.columns:
    if np.issubdtype(pheno_df[df_ndx].dtype, np.number):
        pheno_df[df_ndx] -= pheno_df[df_ndx].mean()

# use patsy to create the design matrix
design = patsy.dmatrix(model_dict["model"], pheno_df, NA_action='raise')
column_names = design.design_info.column_names

print('Model terms: {0}'.format(', '.join(column_names)))

199 rows in design matrix
Model terms: Intercept, sex[T.M], age


In [ ]:
create_flame_model_files()

In [25]:
%%capture 
# create contrasts
if model_dict["contrasts"]:
    contrast_dict = {}
    t_num_contrasts = 0

    for k in model_dict["contrasts"]:
        t_num_contrasts += 1
        try:
            contrast_dict[k] = [n if n != -0 else 0
                                for n in design.design_info.linear_constraint(k).coefs[0]]
        except patsy.PatsyError as e:
            if 'token' in e.message:
                print("A token in contrast \'{0}\' could not be found, should only include tokens from {1}".format(
                        k, ','.join(column_names)))
            raise
else:
     raise ValueError('Model file {0} is missing contrasts'.format(model_file))

num_subjects = len(t_file_list)
t_mat_file, t_grp_file, t_con_file, t_fts_file = create_flame_model_files(design, column_names,
                                                                              contrast_dict, None, [],
                                                                              None, [1] * num_subjects,
                                                                              "Treatment",
                                                                              "randomise_pipe_model",
                                                                              [], "/home/jovyan/work/test_data/")

#return t_file_list, t_num_contrasts, t_mat_file, t_con_file

TypeError: Mismatch between array dtype ('float64') and format specifier ('%1.5e	%1.5e	%1.5e')

In [ ]:
create_flame_model_files()

In [ ]:
create_flame_model_files(design, column_names, contrast_dict, None, [], None, [1] * num_subjects,
                                                                              "Treatment",
                                                                              "randomise_pipe_model",
                                                                              [], model_files_outdir)


In [ ]:
test = {"contrasts": ["+age", "-age", "sex[T.M]", "sex[T.F]"], "model": "sex+age"}


In [ ]:
test = "sex[T.M]".encode('ascii')

In [ ]:
type(test)

In [ ]:
design.design_info.linear_constraint